In [ ]:
import cv2
import numpy as np

# Load the main image
image = cv2.imread('Finding/1.jpg')
original_image = image.copy()  # Keep a copy for drawing results

# Define the 15 objects with their templates, colors (BGR), and shape hints
objects = [
    {'name': 'ice_cream', 'template': 'ice_cream.png', 'color': (0, 255, 255), 'shape': 'cone'},  # Yellow
    {'name': 'sailboat', 'template': 'sailboat.png', 'color': (255, 0, 0), 'shape': 'triangle'},   # Blue with red sail
    {'name': 'toy_car', 'template': 'toy_car.png', 'color': (128, 128, 128), 'shape': 'rectangle'}, # Gray
    {'name': 'red_balloon', 'template': 'red_balloon.png', 'color': (0, 0, 255), 'shape': 'circle'}, # Red
    {'name': 'pegasus', 'template': 'pegasus.png', 'color': (128, 128, 128), 'shape': 'complex'},    # Gray
    {'name': 'watermelon', 'template': 'watermelon.png', 'color': (0, 0, 255), 'shape': 'wedge'},   # Red flesh
    {'name': 'toy_train', 'template': 'toy_train.png', 'color': (147, 20, 255), 'shape': 'rectangle'}, # Pink
    {'name': 'teddy_bear', 'template': 'teddy_bear.png', 'color': (42, 42, 165), 'shape': 'complex'},  # Brown
    {'name': 'grapes', 'template': 'grapes.png', 'color': (0, 255, 255), 'shape': 'cluster'},       # Yellow
    {'name': 'red_bow_tie', 'template': 'red_bow_tie.png', 'color': (0, 0, 255), 'shape': 'bow'},   # Red
    {'name': 'cake_slice', 'template': 'cake_slice.png', 'color': (42, 42, 165), 'shape': 'wedge'}, # Brown
    {'name': 'tennis_ball', 'template': 'tennis_ball.png', 'color': (0, 255, 0), 'shape': 'circle'}, # Green
    {'name': 'strawberry', 'template': 'strawberry.png', 'color': (0, 0, 255), 'shape': 'teardrop'}, # Red
    {'name': 'bunny', 'template': 'bunny.png', 'color': (255, 255, 255), 'shape': 'complex'},       # White
    {'name': 'rubber_duck', 'template': 'rubber_duck.png', 'color': (0, 165, 255), 'shape': 'complex'} # Orange
]

# Function for template matching
def template_matching(image, template_path, threshold=0.7):
    template = cv2.imread(template_path, 0)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    w, h = template.shape[::-1]
    
    res = cv2.matchTemplate(gray_image, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(res >= threshold)
    
    detections = []
    for pt in zip(*loc[::-1]):
        detections.append((pt[0], pt[1], pt[0] + w, pt[1] + h))
    return detections

# Function for color filtering and edge-based detection
def color_and_edge_detection(image, color, shape, color_tolerance=10):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define color range in HSV (example mappings, adjust as needed)
    if color == (0, 255, 255):  # Yellow
        lower = np.array([20, 100, 100])
        upper = np.array([30, 255, 255])
    elif color == (255, 0, 0):  # Blue
        lower = np.array([110, 100, 100])
        upper = np.array([130, 255, 255])
    elif color == (0, 0, 255):  # Red
        lower = np.array([0, 100, 100])
        upper = np.array([10, 255, 255])
    else:  # Generic range, adjust for other colors
        lower = np.array([max(0, c - color_tolerance) for c in color])
        upper = np.array([min(255, c + color_tolerance) for c in color])
        lower = cv2.cvtColor(np.uint8([[lower]]), cv2.COLOR_BGR2HSV)[0][0]
        upper = cv2.cvtColor(np.uint8([[upper]]), cv2.COLOR_BGR2HSV)[0][0]
    
    mask = cv2.inRange(hsv, lower, upper)
    edges = cv2.Canny(mask, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detections = []
    for contour in contours:
        if len(contour) > 5:  # Filter small noise
            x, y, w, h = cv2.boundingRect(contour)
            # Basic shape check (customize further)
            if shape == 'circle' and abs(w - h) < 10:
                detections.append((x, y, x + w, y + h))
            elif shape == 'triangle' and len(cv2.approxPolyDP(contour, 0.04 * cv2.arcLength(contour, True), True)) == 3:
                detections.append((x, y, x + w, y + h))
            elif shape == 'rectangle' and abs(w - h) > 10:
                detections.append((x, y, x + w, y + h))
            # Add more shape checks as needed
    return detections

# Process each object
for obj in objects:
    print(f"Detecting {obj['name']}...")
    
    # Step 1: Template matching
    template_detections = template_matching(image, obj['template'])
    for (x1, y1, x2, y2) in template_detections:
        cv2.rectangle(original_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(original_image, obj['name'], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Step 2: Color and edge detection
    edge_detections = color_and_edge_detection(image, obj['color'], obj['shape'])
    for (x1, y1, x2, y2) in edge_detections:
        # Avoid duplicate detections (simple overlap check)
        overlap = False
        for tx1, ty1, tx2, ty2 in template_detections:
            if abs(tx1 - x1) < 20 and abs(ty1 - y1) < 20:
                overlap = True
                break
        if not overlap:
            cv2.rectangle(original_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(original_image, obj['name'], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Save and display the result
cv2.imwrite('detected_objects.png', original_image)
cv2.imshow('Detected Objects', original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Detecting ice_cream...


AttributeError: 'NoneType' object has no attribute 'shape'